In [1]:
from classes.NetworkXGraph import Graph

In [2]:
# Building graph

    # source: https://graphchallenge.mit.edu/data-sets
    # file_source: https://graphchallenge.s3.amazonaws.com/snap/as20000102/as20000102_adj.tsv
    # file_format: Adjacency TSV (Tab-Separated Values)
        # <dataset-name>_adj.tsv
        # (Row, Col, Value) tuple describing the adjacency matrix of the graph in tab separated format.
        # Adjacency matrix is of size Num_vertices x Num_vertices
    # file: as20000102_adj.tsv (Autonomous Systems graph from January 02 2000)
    # num of nodes:      6.474
    # num of edges:     25.144
    # num of triangles:  6.584

g = Graph()
g.build_from_tsv('../datasets/as20000102_adj.tsv')

Name: 
Type: Graph
Number of nodes: 6474
Number of edges: 12572
Average degree:   3.8838
Number of lines: 25144
Number of triangles:  6584
Clustering coefficient:  0.2522224351161442
Diameter:  9
elapsed time in seconds: 385.192646


In [3]:
# Using samples to build graphs and to calculate

g.get_analisys_from_samples(['clustering_coefficient', 'diameter'])


NetworkXError: Found infinite path length because the graph is not connected

In [ ]:
g.plot_analisys(['clustering_coefficient', 'diameter'])